In [20]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import torch 
import numpy as np
import pandas as pd 

In [21]:
df = pd.read_csv('../data/labels.csv')
print(df.head())

         w        h       p  d=p/h  f=h/w      w.1      h.1     p.1
0  1160.51   724.09  234.81   0.32   0.62  1160.51   724.09  234.81
1   999.46  1163.58  134.61   0.12   1.16   999.46  1163.58  134.61
2  6220.51  1478.29     NaN   0.00   0.24  6220.51  1478.29     NaN
3  1197.39  1429.12     NaN   0.00   1.19  1197.39  1429.12     NaN
4   973.34   678.29  204.69   0.30   0.70   973.34   678.29  204.69


In [22]:
image_dir='../data/images'
image_files=sorted([f for f in os.listdir(image_dir) if f.endswith('.tif')])
print(f"Found {len(image_files)} .tif images.")

Found 48 .tif images.


In [23]:
image_tensors=[] 
for filename in image_files: 
    img_path = os.path.join(image_dir, filename) 
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) 
    img_resized = cv2.resize(img, (224,224)) 
    img_normalized = img_resized.astype(np.float32) / 255.0 
    img_tensor = torch.from_numpy(img_normalized).unsqueeze(0)
    image_tensors.append(img_tensor)
print("Total processed images:", len(image_tensors))
print("Example shape:", image_tensors[0].shape)
print("Pixel range:", image_tensors[0].min().item(), "→", image_tensors[0].max().item())

Total processed images: 48
Example shape: torch.Size([1, 224, 224])
Pixel range: 0.054901961237192154 → 0.8901960849761963


In [32]:
labels_path = '../data/labels.csv'
df = pd.read_csv(labels_path)
df = df[['d=p/h', 'f=h/w', 'w', 'h', 'p']]

print(df.head())

   d=p/h  f=h/w        w        h       p
0   0.32   0.62  1160.51   724.09  234.81
1   0.12   1.16   999.46  1163.58  134.61
2   0.00   0.24  6220.51  1478.29     NaN
3   0.00   1.19  1197.39  1429.12     NaN
4   0.30   0.70   973.34   678.29  204.69


In [40]:
label_tensors = []
for i in range(len(df)):
    row = df.iloc[i].values.astype(np.float32)
    label_tensor = torch.tensor(row)
    label_tensors.append(label_tensor)
print("Total labels:", len(label_tensors))
print("Example label:", label_tensors[0])
print("Shape:", label_tensors[0].shape)


Total labels: 48
Example label: tensor([3.2000e-01, 6.2000e-01, 1.1605e+03, 7.2409e+02, 2.3481e+02])
Shape: torch.Size([5])


In [8]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
from dataset import MicrostructureDataset
from torch.utils.data import DataLoader

In [9]:
dataset = MicrostructureDataset(
    image_dir='../data/images',
    label_csv='../data/labels.csv'
)
dataloader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)
for batch in dataloader:
    images, labels = batch
    print("Images batch shape:", images.shape)
    print("Labels batch shape:", labels.shape)
    break


Images batch shape: torch.Size([8, 1, 224, 224])
Labels batch shape: torch.Size([8, 5])


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

c:\Users\aryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
import sys, os
sys.path.append(os.path.abspath('../src'))

from model import MicrostructureCNN
dummy_input = torch.randn(8, 1, 224, 224)  # Simulate batch of 8 grayscale images
model = MicrostructureCNN() 
output = model(dummy_input)
print("Output shape:", output.shape)

Output shape: torch.Size([8, 5])
